# Scale response spectrum

In [2]:
import pandas as pd
import numpy as np
import os 
import sys
import comtypes.client

## Input names of load cases here
Run the rest of the code

In [27]:
# user input

rs_x_name = 'RS X'
rs_y_name = 'RS Y'
elf_x_name = 'Seismic ELF X'
elf_y_name = 'Seismic ELF Y'

load_cases = [rs_x_name, rs_y_name, elf_x_name, elf_y_name]

# Initialization of ETABSObject
# create API helper object
helper = comtypes.client.CreateObject("ETABSv1.Helper")
helper = helper.QueryInterface(comtypes.gen.ETABSv1.cHelper)
print("Succesfully created helper object")

try:
    #get the active ETABS object
    ETABSObject = helper.GetObject("CSI.ETABS.API.ETABSObject") 
    SapModel = ETABSObject.SapModel
    print("Succesfully created SapModel")
except (OSError, comtypes.COMError):  
    print("No running instance of the program found or failed to attach.")
    sys.exit(-1)


SapModel.Analyze.RunAnalysis()

for i in load_cases:
    SapModel.Results.Setup.SetCaseSelectedForOutput(i, True)

base_reactions = SapModel.Results.BaseReact()

load_cases_output = base_reactions[1]
fx_output = base_reactions[4]
fy_output = base_reactions[5]

fx_elf = fx_output[load_cases_output.index(elf_x_name)]
fy_elf = fy_output[load_cases_output.index(elf_y_name)]
fx_rs = fx_output[load_cases_output.index(rs_x_name)]
fy_rs = fy_output[load_cases_output.index(rs_y_name)]

sf_x = abs(fx_elf/fx_rs)
sf_y = abs(fy_elf/fy_rs)

rs_x_data = SapModel.LoadCases.ResponseSpectrum.GetLoads(rs_x_name)
rs_y_data = SapModel.LoadCases.ResponseSpectrum.GetLoads(rs_y_name)
#rs_x_data[3][0] = rs_x_data[3][0]*sf_x
#rs_y_data[3][0] = rs_y_data[3][0]*sf_y
rs_x_data[3] = tuple(np.array(rs_x_data[3])*sf_x)
rs_y_data[3] = tuple(np.array(rs_y_data[3])*sf_y)

retx = SapModel.LoadCases.ResponseSpectrum.SetLoads(rs_x_name,rs_x_data[0],rs_x_data[1],rs_x_data[2],rs_x_data[3],rs_x_data[4],rs_x_data[5])
rety = SapModel.LoadCases.ResponseSpectrum.SetLoads(rs_y_name,rs_y_data[0],rs_y_data[1],rs_y_data[2],rs_y_data[3],rs_y_data[4],rs_y_data[5])
SapModel.Analyze.RunAnalysis()

print(f'X scaled to : {sf_x}')
print(f'Y scaled to : {sf_y}')

Succesfully created helper object
Succesfully created SapModel
X scaled to : 0.9999999999999998
Y scaled to : 0.9999999999999999
